In [1]:
import os, random, time, io
from pathlib import Path
from typing import List

import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, Subset, Dataset
from torchvision import datasets, transforms, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, accuracy_score
import kagglehub





c:\Users\angus\OneDrive - The University of Sydney (Students)\Classes\Python\ENGG 2112\ML PROJECT\ENNG-2112\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(torch.__version__)          # e.g., 2.x.x+cu124
print("Built with CUDA:", torch.version.cuda)
print("CUDA available:", torch.cuda.is_available())

2.6.0+cu124
Built with CUDA: 12.4
CUDA available: True


In [22]:
# Reproducibility + device
SEED = 56
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cpu


In [7]:

# Check if a graphics card (GPU) is available for faster training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device being used:", device)

Device being used: cpu


In [ ]:
# Download dataset (Kaggle)
# Dataset: alistairking/recyclable-and-household-waste-classification
print("Downloading dataset...")
ds_path = kagglehub.dataset_download(
    "alistairking/recyclable-and-household-waste-classification"
)
print("Dataset downloaded to:", ds_path)


100%|██████████| 920M/920M [01:39<00:00, 9.65MB/s] 

Extracting files...


Dataset downloaded to: C:\Users\angus\.cache\kagglehub\datasets\alistairking\recyclable-and-household-waste-classification\versions\1


In [11]:
# Some Kaggle datasets have extra folders like "images/images"
# This loop finds the real folder that contains all the image categories.
candidates = [
    Path(ds_path) / "images" / "images",
    Path(ds_path) / "images",
    Path(ds_path),
]

DATA_ROOT = None
for c in candidates:
    if c.exists() and any(d.is_dir() for d in c.iterdir()):
        DATA_ROOT = c
        break

if DATA_ROOT is None:
    raise FileNotFoundError(f"Could not find image folder in: {path}")

print("Image root folder found:", DATA_ROOT)

Image root folder found: C:\Users\angus\.cache\kagglehub\datasets\alistairking\recyclable-and-household-waste-classification\versions\1\images\images


In [12]:
# Build full ImageFolder (PNG-only filter)
png_only = lambda p: str(p).lower().endswith(".png")
full_ds = datasets.ImageFolder(root=str(DATA_ROOT), transform=None, is_valid_file=png_only)
class_names: List[str] = full_ds.classes
num_classes = len(class_names)
print(f"Classes ({num_classes}):", class_names[:10], "..." if num_classes > 10 else "")
print("Total PNG images:", len(full_ds.samples))

Classes (30): ['aerosol_cans', 'aluminum_food_cans', 'aluminum_soda_cans', 'cardboard_boxes', 'cardboard_packaging', 'clothing', 'coffee_grounds', 'disposable_plastic_cutlery', 'eggshells', 'food_waste'] ...
Total PNG images: 15000


In [13]:
# --- Check what categories (folders) exist ---
# Each folder inside the dataset represents one type of waste.
class_dirs = sorted([d.name for d in DATA_ROOT.iterdir() if d.is_dir()])
print("Number of categories found:", len(class_dirs))
print("Example categories:", class_dirs[:10])

Number of categories found: 30
Example categories: ['aerosol_cans', 'aluminum_food_cans', 'aluminum_soda_cans', 'cardboard_boxes', 'cardboard_packaging', 'clothing', 'coffee_grounds', 'disposable_plastic_cutlery', 'eggshells', 'food_waste']


In [14]:
# --- Define image transformations (resizing and data augmentation) ---
# These changes help prepare the photos before training the model.

IMG_SIZE = 224  # final image size (in pixels)

# Training transformations (adds small random changes for variety)
train_tf = transforms.Compose([
    transforms.Resize((256, 256)),             # make all images similar size
    transforms.RandomResizedCrop(IMG_SIZE,     # randomly crop and resize
                                 scale=(0.8, 1.0),
                                 ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),         # randomly flip images left-right
    transforms.RandomRotation(10),             # rotate slightly (±10°)
    transforms.ColorJitter(                    # small colour adjustments
        brightness=0.10, contrast=0.10, saturation=0.10, hue=0.05),
    transforms.ToTensor(),                     # turn image into numeric array
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # standard colour scaling
                         std=[0.229, 0.224, 0.225]),
])


In [15]:
# Validation/testing transformations (simpler, no random changes)
eval_tf = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# Filter function: only include .png or .PNG images
png_only = lambda p: str(p).lower().endswith(".png")

In [16]:
# --- Load the dataset ---
# Organises images into labelled groups using their folder names.
full_ds = datasets.ImageFolder(
    root=str(DATA_ROOT),
    transform=None,          # we'll apply transforms later
    is_valid_file=png_only   # only include PNG files
)


In [17]:
# Print a quick summary
num_classes = len(full_ds.classes)
print(f"Total number of categories: {num_classes}")
print("First few category names:", full_ds.classes[:10])
print("Total number of images:", len(full_ds.samples))

Total number of categories: 30
First few category names: ['aerosol_cans', 'aluminum_food_cans', 'aluminum_soda_cans', 'cardboard_boxes', 'cardboard_packaging', 'clothing', 'coffee_grounds', 'disposable_plastic_cutlery', 'eggshells', 'food_waste']
Total number of images: 15000
